In [24]:
import tensorflow.keras as keras
import tensorflow as tf

import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Probabilistic ML 00

## Exercise 01

In human psychology, we often use IQ ([intelligence quotient](https://en.wikipedia.org/wiki/Intelligence_quotient)) as a measure of intelligence. IQ is normally distributed with mean 100 and standard deviation (SD) of 15. 

Let's assume that we know SD of IQ, but we don;t know the mean. 

We'll build a probabilistic model that will help us find the true mean of IQ.


In [19]:
# Define distribution params
true_mean = 100.
true_sd = 15.

In [20]:
# Create some data
iq_data = tf.random.normal((10000, 1), true_mean, true_sd)

In [21]:
# Sanity check - are the values correct?
iq_data.numpy().mean(), iq_data.numpy().std()

(100.00762, 15.070722)

In [40]:
# Let's build the model!
model = keras.Sequential([
                          keras.layers.Dense(1, input_shape = (1,)),
                          tfpl.DistributionLambda(
                              lambda m: tfd.Normal(loc = m, scale = true_sd)
                          )
])

Note that our model returns a distribution. We need to build a cost function that reflects that.

In [41]:
# Now, we need a cost function
def neg_loglik(y_true, y_pred):
  return - y_pred.log_prob(y_true)

In [42]:
# Compile the model
model.compile(loss = neg_loglik, optimizer = keras.optimizers.RMSprop())

In [43]:
# Train the model
model.fit(iq_data, iq_data, epochs = 10)

Epoch 1/10
313/313 [==============================] - 0s 709us/step - loss: 86.7388
Epoch 2/10
313/313 [==============================] - 0s 757us/step - loss: 61.7426
Epoch 3/10
313/313 [==============================] - 0s 715us/step - loss: 41.1875
Epoch 4/10
313/313 [==============================] - 0s 728us/step - loss: 25.1760
Epoch 5/10
313/313 [==============================] - 0s 738us/step - loss: 13.6298
Epoch 6/10
313/313 [==============================] - 0s 746us/step - loss: 6.5606
Epoch 7/10
313/313 [==============================] - 0s 745us/step - loss: 3.8170
Epoch 8/10
313/313 [==============================] - 0s 742us/step - loss: 3.6272
Epoch 9/10
313/313 [==============================] - 0s 758us/step - loss: 3.6272
Epoch 10/10
313/313 [==============================] - 0s 697us/step - loss: 3.6271


In [50]:
# Let's see the results!
model(iq_data).mean().numpy().mean()

99.94286

Great, we're very close to the original mean of 100!